In [1]:
%reset -f
import ipyparallel
import os
rc = ipyparallel.Client(profile="mpi")
view = rc[:]
view.apply(os.chdir, os.getcwd())
view['stop'] = True
print view

view.block=True

<DirectView [0, 1, 2, 3,...]>


In [10]:
%%px --block
from mpi4py import MPI
mpi = MPI.COMM_WORLD
bcast = mpi.bcast
barrier = mpi.barrier
rank = mpi.rank


# Engines Imports
import sys
#!pwd   # Replace sys.path.append with this path
sys.path.append('/home/user/JorgeMijares/air-water-vv/2d/Interactive/Temp_Flume_GUI')

import proteus
## Required imports
from proteus.iproteus import * 
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread
import tempFlume as plant
import tempFlume_so as plant_so
from proteus import default_n, default_s, default_so, Comm, Context
print "MPI rank: %i/%i" % (mpi.rank,mpi.size)

[stdout:0] MPI rank: 13/20
[stdout:1] MPI rank: 2/20
[stdout:2] MPI rank: 7/20
[stdout:3] MPI rank: 17/20
[stdout:4] MPI rank: 5/20
[stdout:5] MPI rank: 8/20
[stdout:6] MPI rank: 10/20
[stdout:7] MPI rank: 18/20
[stdout:8] MPI rank: 14/20
[stdout:9] MPI rank: 1/20
[stdout:10] MPI rank: 0/20
[stdout:11] MPI rank: 4/20
[stdout:12] MPI rank: 3/20
[stdout:13] MPI rank: 16/20
[stdout:14] MPI rank: 12/20
[stdout:15] MPI rank: 15/20
[stdout:16] MPI rank: 6/20
[stdout:17] MPI rank: 11/20
[stdout:18] MPI rank: 19/20
[stdout:19] MPI rank: 9/20


In [11]:
# Only Kernel Imports
import parUtilities
reload(parUtilities)
import pandas as pd
from IPython.display import clear_output

# Kernel and Engines imports
#with view.sync_imports():  

    

## Simulation Parameters

In [12]:

#Parameters
dt=0.1            # Not sure if disabled, more testing needed, assign same value as dtOut too
cfl=0.75          # Not working yet, modify too in tempflume.py
dtOut=0.1          # Modify nDTout
T=0.2             # Total Simulation Time
Tank_he=0.03        # Domain he
Caisson_he=0.03     # Interpolation distance of caisson perimeter
enSphere=False        # Replace Caisson with sphere for testing of parameters
radius=0.1


## Update Context

In [18]:

#Experimental Case

Scales = [21,25]        # Scale could be 21 or 25
index  = 44             #Refer to DoE_TempFlume.xlsx to check case conditions
scale  = 21
#for s_ind, scale in enumerate(Scales):
if True:
# Base Model, replace the model with tempFlume_test for testing
# tempFlume is the most stable version until now.
        xl    = pd.ExcelFile("DoE_TempFlume.xlsx")
        df1   = xl.parse(0)
        df2   = xl.parse(1)
        df3   = xl.parse(2)
        df4   = xl.parse(3, header=0, index_col=0)
        df5   = xl.parse(4, header=0, index_col=0)
        

        clear_output()

        reload(plant)
        reload(plant_so)

        if scale == 21:
            df_DoE  = df4
            df_data = df2

        elif scale==25:
            df_DoE  = df5
            df_data = df3
        else:
            print 'Wrong Scale'
            () + 1
        #for index in range(view.pull('df4', targets=0).shape[0]):
        if True:
            test_name     = 'Results/S{0}T{1}'.format(scale,index)
            status        = df_DoE['Finished'].tolist()[index]
            #if status == 'Completed':
            #    continue
            
            # Read Index from DoE table
            iFlow   = df_DoE['Flow Speed'].tolist()[index]
            iDepth  = df_DoE['Depth'].tolist()[index]
            iOffset = df_DoE['Offset'].tolist()[index]
            
            # Read Tag parameters from table
            tagFlow   = df_data['Speed Tags'].tolist()[iFlow]
            tagDepth  = df_data['Depth Tags'].tolist()[iDepth]
            tagOffset = df_data['Offset Tags'].tolist()[iOffset]
            
            # Read parameters from table
            flowSpeed  = df_data['Flow Speed'].tolist()[iFlow]
            waterLevel = df_data['Depth'].tolist()[iDepth]
            offset     = df_data['Offset'].tolist()[iOffset]
            
            #Change Context
            view['plant.opts.water_level'] = waterLevel
            view['plant.opts.tank_dim']    = (3.0,waterLevel+0.5)

            # plant.opts.wind_velocity=(flowSpeed,0.)
            view['plant.opts.inflow_velocity'] = flowSpeed
            # plant.opts.outflow_velocity=flowSpeed
            view['plant.opts.caisson_scale']   = float(scale)
            view['plant.opts.caisson_Yoffset'] = -offset
            
            view['plant.opts.dtOut']        = dtOut
            view['plant.opts.sphere']       = enSphere
            view['plant.opts.sphereRadius'] = radius
            view['plant.opts.dt_fixed']     = dt
            view['plant.opts.cfl']          = cfl
            view['plant.opts.T']            = T
            view['plant.opts.he']           = Tank_he
            view['plant.opts.he_caisson']   = Caisson_he

            print 'Speed={0}, Depth={1}, Offset={2}, Scale={3}'.format(
                                                flowSpeed,
                                                waterLevel,
                                                offset,
                                                scale)
            def Setup_Simulation():
                global ct,ns, simulation_thread
                plant.Update_Model()
                Context.setFromModule(plant,mutable=True)
                ct = Context.get()
                plant_so.ct =ct
                
                so = plant_so
                so.tnList = [0.0]+[i*plant.opts.dtOut for i in range(1,plant.nDTout+1)]        
                info = open("TimeList.txt","w")
                for time in so.tnList:
                    info.write(str(time)+"\n")
                info.close()
                plant_so.tnList=so.tnList
                pList=[]
                nList=[]
                so.sList=[]
                OptDB = PETSc.Options()
                for (p,n) in so.pnList:
                    so.sList.append(default_s)
                    pList.append(__import__(p))
                    reload(pList[-1])
                    nList.append(__import__(n))
                    reload(nList[-1])
                    pList[-1].name = p
                    nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
                    nList[-1].levelLinearSolver = default_n.KSP_petsc4py
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
                    OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
                opts.save_dof = True
                opts.dataDir='.'
                opts.probDir='.'
                opts.logLevel=7
                opts.verbose=True
                opts.viewMesh=True


                ## Create numerical solution
                ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)


                ## Start Simulation
                simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
                simulation_thread.start()
                
            view.apply_sync(Setup_Simulation)
            print 'Simulation Started'
            ## Monitor Output
            parUtilities.view=view
            error=parUtilities.monitor_simulation(refresh=5.0)      
            

Speed=1.19723063299, Depth=0.0725714285714, Offset=0.0164112, Scale=21
Simulation Started


CompositeError: one or more exceptions from call to method: _pull
[0:apply]: PicklingError: Can't pickle <type 'module'>: attribute lookup __builtin__.module failed
[1:apply]: PicklingError: Can't pickle <type 'module'>: attribute lookup __builtin__.module failed
[2:apply]: PicklingError: Can't pickle <type 'module'>: attribute lookup __builtin__.module failed
[3:apply]: PicklingError: Can't pickle <type 'module'>: attribute lookup __builtin__.module failed
.... 16 more exceptions ...

In [ ]:
%%px --targets 1


import os   
print test_name
checkpath=os.path.exists('./'+test_name) 
if not checkpath:
        os.mkdir(test_name)
with open(test_name+'/opts.txt', 'w+') as fp:
    fp.write('\n'.join('%s %s' % (key , val) for key, val in plant.opts._asdict().items()))


!cp tempFlume_p.xmf ./{test_name}/tempFlume_p.xmf
!cp tempFlume_p.h5  ./{test_name}/tempFlume_p.h5

if ns.systemStepController.converged():
    error =False
else:
    error = True
    
if error:
    !cp forceHistory_p.txt ./{test_name}/forceHistory_p.txt
    !cp forceHistory_v.txt  ./{test_name}/forceHistory_v.txt
    Exp_Status='Completed'
else:
    Exp_Status='Crashed'
    
print Exp_Status

#!rm ./Results/{ExpTag}_p.xmf
#!rm ./Results/{ExpTag}_p.h5
#!cp tempFlume_p.xmf ./Results/{ExpTag}_p.xmf
#!cp tempFlume_p.h5  ./Results/{ExpTag}_p.h5
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('DoE_TempFlume.xlsx', engine='xlsxwriter')

if scale==21:
    df4.loc[index,'Finished']=Exp_Status
    df4.loc[index,'Time']=ns.systemStepController.t_system

elif scale==25:
    df5.loc[index,'Finished']=Exp_Status
    df5.loc[index,'Time']=ns.systemStepController.t_system
else:
    print 'Scale Error'

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer, sheet_name='Parameters')
df2.to_excel(writer, sheet_name='Parameters Scale 1-21')
df3.to_excel(writer, sheet_name='Parameters Scale 1-25')
df4.to_excel(writer, sheet_name='Tests Scale 1-21')
df5.to_excel(writer, sheet_name='Tests Scale 1-25')
# Close the Pandas Excel writer and output the Excel file.
writer.save()